In [1]:
from sentence_transformers import SentenceTransformer, util

e:\AI Projects\Hugging Face LLaMa recipes\huggingface-llama-recipes\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Step 1: Define a dictionary with text snippets

text_snippets = {
    1: "Fiona thanked Ethan for his unwavering support and promised to cherish their friendship.",
    2: "As they ventured deeper into the forest, they encountered a wide array of obstacles.",
    3: "Ethan and Fiona crossed treacherous ravines using rickety bridges, relying on each other's strength.",
    4: "Overwhelmed with joy, Fiona thanked Ethan and disappeared into the embrace of her family.",
    5: "Ethan returned to his cottage, heart full of memories and a smile brighter than ever before."
}

In [3]:
# Step 2: Initialize the embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

e:\AI Projects\Hugging Face LLaMa recipes\huggingface-llama-recipes\venv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Step 3: Embed all text snippets
snippets_embedded = {key: model.encode(text) for key, text in text_snippets.items()}


In [5]:
# Step 4: Function to retrieve the closest matching snippet using cosine similarity
def retrieve_snippet(query, top_k=1):
    query_embedded = model.encode(query)
    similarities = {key: util.pytorch_cos_sim(query_embedded, snippet_emb).item() 
                    for key, snippet_emb in snippets_embedded.items()}
    
    # Sort by similarity score
    closest_matches = sorted(similarities.items(), key=lambda item: item[1], reverse=True)[:top_k]
    retrieved_texts = [text_snippets[key] for key, _ in closest_matches]
    return retrieved_texts

In [6]:
# Step 5: Generate a new prompt with the retrieved text
def generate_prompt(query):
    retrieved_texts = retrieve_snippet(query)
    prompt = f"Context: {' '.join(retrieved_texts)}\nQuestion: {query}\nAnswer: "
    return prompt

In [7]:
# Step 6: Ask a question
query = "Why did Fiona thank Ethan?"
prompt = generate_prompt(query)
print(prompt)

Context: Fiona thanked Ethan for his unwavering support and promised to cherish their friendship.
Question: Why did Fiona thank Ethan?
Answer: 
